In [36]:
import psycopg2
import mysql.connector
from mysql.connector.constants import ClientFlag
import zipfile
import os
import random
import shutil
import pandas as pd
import json
import time
import datetime
import glob
from collections import namedtuple
import numpy as np

## Functions to process data 

In [2]:
#function

Author = namedtuple('Author', 'guid name email source location imageUrl coverUrl')
Visual=namedtuple('Visual', 'guid type contentType generatingRenditions url caption credit language')
Article=namedtuple('Article', 'guid templateName availableInPreview url creationDate modificationDate title channel lead chapters')

def process_analytic_file(thisPath,thisFile):
    """
    this function specifically transforms the inputs of the analytic file of GLU7027
    it transforms the dict structure into tuples and performs the following modifs
    
    1;split the name into 2 parts  product-->the newspaper's name|plateform
    2.transform created_at date into a mysql readable format
    3.transforms the name into an integer value ex: View60-->60
    """
    path=thisPath+thisFile
    with open(path) as jar:
        data = json.load(jar)
        output=[]
        for d in data:
            d['createdAt']=d['createdAt'][:-1]
            temp=d['product'].split('/') # to create additional column for the type of platform
                
            d['product']=temp[0]
            d['interface']=temp[1]
            #just get the number part the "view" has no purpose
            if len(d['name'])==4:
                d['name']=0
            elif len(d['name'])==5:
                d['name']=int(d['name'][-1:])
            else:
                d['name']=int(d['name'][-2:])
        jar.close() 
        return [tuple(d.values()) for d in data]
    

def process_article_file(thisPath,thisFile):
    """
    this function transform the JSON of an article into 3 namedTuples, one for the article,the author and
    the visuals in the article.
    
    this function processes one article at a time
    
    inputs:it uses the folder path and file name as inputs
    outputs: a list of Authors namedTuples
             a list of Visual named tuples
             a list of Articles named tuples

    
    author and visual use the guid of the article as its foreign key
    Because it is a nested json and not all json have all the information, exception handling was necessary
    to avoid the pyramid of doom
    
    """
    path=thisPath+thisFile
    with open(path) as jar:
        data = json.load(jar)
        articles=[]
        authors=[]
        visuals=[]
        
        #author part:
        for author in data['authors']:
           
            authorKeys=list(author.keys())
            guid=data['id'] #it will be it foreign key to link back article to author
            
            if 'name' in authorKeys:
                name=author['name']
            else:
                name='Elvis Gratton'
            if 'email' in authorKeys:
                email=author['email']
            else:
                email="bobGratton@garabeGratton.com"
            if 'source' in authorKeys:
                source=author['source']
            else:
                source=''
            if 'location' in authorKeys:
                location=author['location']
            else:
                location='Lacheneuil'
            if 'imageUrl' in authorKeys:
                imageUrl=author['imageUrl']
            else:
                imageUrl=''
            if 'coverUrl' in authorKeys:
                coverUrl=author['coverUrl']
            else:
                coverUrl=''
            authors.append(Author(guid,name,email,source,location,imageUrl,coverUrl))
            
        #visual part
        visualKeys=list(data['visual'].keys())
      
        guid= data['id']
        
        if 'type'in visualKeys:
            type_= data['visual']['type']
        else:
            type_=''
        
        if 'contentType' in visualKeys:
            contentType=data['visual']['contentType']
        else:
            contentType=''
        
        if 'generatingRenditions' in visualKeys:
            generatingRenditions=data['visual']['generatingRenditions']
        else:
            generatingRenditions=False
            
        if 'url' in visualKeys:
            url=data['visual']['url']
        else:
            url=''
        if 'caption' in visualKeys:
            try:
                caption=data['visual']['caption']['fr']
            except:
                caption=''
        else:
            caption=''
        if 'credit' in visualKeys:
            #credit and language are nested one level deeper but it is always a 1 key 1 value dict
            try:
                credit =list(data['visual']['credits'].values())[0]
                language=list(data['visual']['credits'].keys())[0]
            except:
                credit=''
                language=''
        else:
            credit=''
            language='' 
                
        visuals.append(Visual(guid,type_, contentType,generatingRenditions,url,caption,credit,language))
                              
            
        #article part 
        articleKeys=data.keys()
        guid=data['id']
        templateName=data['templateName']
            
        availableInPreview=data['availableInPreview']
        url=data['url'] 
        creationDate=data['creationDate'][:-1] #must remove de 'Z' at the end of date fields
        modificationDate=data['modificationDate'][:-1] 
        if 'title' in articleKeys:
            try:
                title=list(data['title'].values())[0]
                
            except:
                title=''
        else:
            title=''
    
        try:
            channel=list(data['channel'].values())[0] 
            
        except:
            channel=''
            
        try:
            lead=list(data['lead'].values())[0]
        except:
            lead=''
        
        if 'chapters' in articleKeys:
            chapters=''
            for paragraph in data['chapters']:
                paragraphKeys= paragraph.keys()
                if 'text' in paragraphKeys:
                    chapters=chapters+'\n'+paragraph['text']['fr']
                
             
        else:
            chapters=[{'abc':'cba'}]
        articles.append(Article(guid,templateName,availableInPreview,url,creationDate,modificationDate,title,channel,lead,chapters))                  
      
            
     
        jar.close() 
        return articles,authors,visuals  

## DataBase connections configuration 

In [84]:
config = {
    'user': 'root',
    'password': 'abc123',
    'host': '34.69.198.118',
    'use_pure':True,
    'database':'GLO_7027',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'server-ca.pem',
    'ssl_cert': 'client-cert.pem',
    'ssl_key': 'client-key.pem'
}

cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

## Database table configuration 

## MySQL Queries 

In [ ]:
"""



# create a new  database
cursor = cnxn.cursor()  # initialize connection cursor
cursor.execute('CREATE DATABASE GLO_7027')  # create a new  database
cnxn.close()  # close connection because we will be reconnecting to testdb


config['database'] = 'GLO_7027'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

#drop a table
cursor.execute("DROP TABLE XXX")
cnxn.commit()

#create analytics table
cursor.execute("CREATE TABLE analytics ("
               "id INT AUTO_INCREMENT PRIMARY KEY,"
               "created_at DATETIME,"
               "guid VARCHAR(50),"
               "hash VARCHAR(255),"
               "product VARCHAR(255),"
               "interface VARCHAR(255) )")

#insert into analytics db
insert_into_analytics = ("INSERT INTO analytics (hash, click_time, product, created_at,interface) "
         "VALUES (%s, %s, %s, %s,%s)")


#create article table
cursor.execute("CREATE TABLE articles ("
               "id INT AUTO_INCREMENT PRIMARY KEY,"
               "guid VARCHAR(50),"
               "templateName TEXT,"
               "availableInPreview BOOL,"
               "url VARCHAR(255),"
               "creationDate DATETIME,"
               "modificationDate DATETIME,"
               "title TEXT,"
               "channel TEXT,"
               "lead TEXT,"
               "articleText TEXT )")

#insert into articles db
insert_into_articles = ("INSERT INTO articles (guid, templateName, availableInPreview, url,creationDate,modificationDate,title,channel,lead,articleText) "
         "VALUES (%s, %s, %s, %s,%s,%s, %s, %s, %s,%s)")


#create authors table

cursor.execute("CREATE TABLE authors ("
               "id INT AUTO_INCREMENT PRIMARY KEY,"
               "guid VARCHAR(50),"
               "name VARCHAR(255),"
               "email VARCHAR(255),"
               "source VARCHAR(255),"
               "location VARCHAR(255),"
               "imageUrl VARCHAR(255),"
               "coverUrl VARCHAR(255) )")

insert_into_authors = ("INSERT INTO authors (guid, name, email, source,location,imageUrl,coverUrl) "
         "VALUES (%s, %s, %s, %s,%s,%s, %s)")



#create visual table
Visual=namedtuple('Visual', 'guid type contentType generatingRenditions url caption credit language')

cursor.execute("CREATE TABLE visuals ("
               "id INT AUTO_INCREMENT PRIMARY KEY,"
               "guid VARCHAR(50),"
               "type VARCHAR(255),"
               "contentType VARCHAR(255),"
               "generatingRenditions BOOLEAN,"
               "url VARCHAR(255),"
               "caption TEXT,"
               "credit VARCHAR(255),"
               "language VARCHAR(255) )")
               
insert_into_visuals = ("INSERT INTO visuals (guid, type, contentType, generatingRenditions,url,caption,credit,language) "
         "VALUES (%s, %s, %s, %s,%s,%s, %s,%s)")

cursor.execute("SELECT COUNT(*) from analytics" )
out=cursor.fetchall()
out

cnxn.commit()

# Inputs 

This section contains key variables used as inputs to create the final data uploaded to SQL databases

In [3]:
#process JSON files

#Inputs for analytics file
start_time = time.process_time()
#if it is the first time
processed_files=[]
#if not then upload the list of already processed files
#processed_files=pd.read_csv('processed_files_lenouvelliste.csv')['file_name'].tolist()
files=os.listdir('/Users/gabounet/ULaval/GLO_2027/TrainSample/')
total=len(files)
#path_to_file='/Users/gabounet/ULaval/GLO_2027/RichardKhoury/Analytics/lavoixdelest/'

# inputs for query
step=50000


# Treatment and upload analytics file

In [22]:
"""
#process JSON files



count=1
for file in files[:]:
    
    if file not in processed_files:
        #try to open the json if it doesn't have a bug
        try:
            output=process_analytic_file(path_to_file,file)

        except:
            output=[]
        #load data to google mysqldb by batch equal to the step to avoid connection drop
        
        start=0
        limit=len(output)
        while start< limit:

            cursor.executemany(insert_into_analytics, output[start:min(limit,start+step)])
            cnxn.commit()  # and commit changes
            start=start+step


        processed_files.append(file)
        print(file," ",count," ouf of ",total)
        count=count+1"""


# Treatment and upload of  Article,Authors and Viz tables

### Inputs 

In [103]:
#process JSON files

#Inputs for analytics file
start_time = time.process_time()
#if it is the first time
processed_files=[]
#if not then upload the list of already processed files
#processed_files=pd.read_csv('processed_files_lenouvelliste.csv')['file_name'].tolist()
files=os.listdir('/Users/gabounet/ULaval/GLO_2027/RichardKhoury/train/')
total=len(files)
#path_to_file='/Users/gabounet/ULaval/GLO_2027/RichardKhoury/Analytics/lavoixdelest/'
correctFiles=[f for f in files if '--pub' not in f] # only files not containing "--pub-infos"
# inputs for query
step=50000
limit=len(correctFiles)
output_path='/Users/gabounet/ULaval/GLO_2027/GLO_2027_work/outputs_to_db/'
input_path='/Users/gabounet/ULaval/GLO_2027/RichardKhoury/train/'

In [109]:

start_time = time.process_time()
n=1
start=0
step=100000
limit=len(correctFiles)

while start< limit:
    art_list=[]
    auth_list=[]
    viz_list=[]
    
    # select file in the range and get date to then store in lists
    for file in correctFiles[start:min(start+step,limit)]:
        art,auth,viz=process_article_file(input_path,file)
        #print(n)
        for article in art:
            art_list.append(article) 
        #for author in auth:
            #auth_list.append(author)
        #for vizu in viz:
            #viz_list.append(vizu)
        n=n+1
        if n%50000==0:
            print(n)
    #increment the starting point
    

    #dump to csv
    #articles
    fileName='articles_data_V2'+'_'+str(start)+'-'+str(min(start+step,limit))+'.csv'
    pd.DataFrame.from_records(art_list,columns=art_list[0]._fields).to_csv(output_path+fileName,index=False)
    start=start+step
    
    #authors
    #fileName='authors_data'+'_'+str(start)+'-'+str(stop)+'.csv'
    #pd.DataFrame.from_records(auth_list,columns=auth_list[0]._fields).to_csv(output_path+fileName,index=False)
    #vizuals
    #fileName='visuals_data'+'_'+str(start)+'-'+str(stop)+'.csv'

    #pd.DataFrame.from_records(viz_list,columns=viz_list[0]._fields).to_csv(output_path+fileName,index=False)
    
    print(time.process_time()-start_time)
    


50000
100000
37.431827999999996
150000
200000
92.58610199999998
250000
300000
146.013508
350000
400000
202.91644799999995
450000
500000
259.8742199999999
550000
310.52235199999996


## process articles files by chunks

In [110]:
files=os.listdir('/Users/gabounet/ULaval/GLO_2027/GLO_2027_work/outputs_to_db/')
filesToProcess=[f for f in files if 'V2' in f]
filesToProcess

['articles_data_V2_300000-400000.csv',
 'articles_data_V2_200000-300000.csv',
 'articles_data_V2_400000-500000.csv',
 'articles_data_V2_500000-592820.csv',
 'articles_data_V2_100000-200000.csv',
 'articles_data_V2_600000-592820.csv']

In [111]:
files=os.listdir('/Users/gabounet/ULaval/GLO_2027/GLO_2027_work/outputs_to_db/')


for file in filesToProcess:
    path='/Users/gabounet/ULaval/GLO_2027/GLO_2027_work/outputs_to_db/'
    fileName=file
    dataToProcess=path+fileName
    insert_into_articles = ("INSERT INTO articles (guid, templateName, availableInPreview, url,creationDate,modificationDate,title,channel,lead) "
             "VALUES (%s, %s, %s, %s,%s,%s, %s, %s, %s)")



    n=0
    for chunk in pd.read_csv(dataToProcess, chunksize=5000):


        #data cleaning
        chunk['availableInPreview'] = chunk['availableInPreview'].fillna(0)
        chunk=chunk.replace(np.nan, '', regex=True)
        del chunk['chapters']
        output_to_db=list(chunk.itertuples(index=False, name=None))
        output_to_db=[output for output in output_to_db if output[0]!='']

        cursor.executemany(insert_into_articles, output_to_db)
        cnxn.commit()  
        n=n+1
        print(n)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
